In [1]:
# Importing libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mstats
import scipy.stats as stats
from scipy.stats import ttest_ind
from sklearn.preprocessing import MinMaxScaler

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [52]:
# Load the dataset
data = pd.read_csv('/kaggle/input/midterm-media-cost-T/train.csv',)
data.pop('id')
# View the first few rows of the data
# pd.set_option('display.max_columns', None) # This is so we can view all columns
print(data.head())
# print(data.OverallQual.value_counts())


     food_category food_department food_family  store_sales(in millions)  \
0  Breakfast Foods    Frozen Foods        Food                      5.52   
1  Breakfast Foods    Frozen Foods        Food                      3.68   
2  Breakfast Foods    Frozen Foods        Food                      4.08   
3  Breakfast Foods    Frozen Foods        Food                      4.08   
4  Breakfast Foods    Frozen Foods        Food                      5.44   

   store_cost(in millions)  unit_sales(in millions)         promotion_name  \
0                   2.5944                        3  Cash Register Lottery   
1                   1.1776                        2  Cash Register Lottery   
2                   1.4280                        3       Double Down Sale   
3                   1.4688                        3       Double Down Sale   
4                   2.5568                        4  Cash Register Lottery   

  sales_country marital_status gender  ...  grocery_sqft frozen_sqft  \
0 

In [36]:
print(data.shape)
data.info()

(49024, 39)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49024 entries, 0 to 49023
Data columns (total 39 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   food_category             49024 non-null  object 
 1   food_department           49024 non-null  object 
 2   food_family               49024 non-null  object 
 3   store_sales(in millions)  49024 non-null  float64
 4   store_cost(in millions)   49024 non-null  float64
 5   unit_sales(in millions)   49024 non-null  int64  
 6   promotion_name            49024 non-null  object 
 7   sales_country             49024 non-null  object 
 8   marital_status            49024 non-null  object 
 9   gender                    49024 non-null  object 
 10  total_children            49024 non-null  int64  
 11  education                 49024 non-null  object 
 12  member_card               49024 non-null  object 
 13  occupation                49024 non-null  object 

In [37]:
duplicate_rows = data[data.duplicated()]
if not duplicate_rows.empty:
    print("重复数据：")
    print(duplicate_rows)
else:
    print("没有重复数据")

没有重复数据


In [9]:
# Get summary statistics of the numerical columns
data.describe()

,store_sales(in millions),store_cost(in millions),unit_sales(in millions),total_children,avg_cars_at home(approx),num_children_at_home,SRP,gross_weight,net_weight,recyclable_package,...,store_sqft,grocery_sqft,frozen_sqft,meat_sqft,coffee_bar,video_store,salad_bar,prepared_food,florist,cost
count,49024.000000,49024.000000,49024.000000,49024.000000,49024.000000,49024.000000,49024.000000,49024.000000,49024.000000,49024.000000,...,49024.000000,49024.000000,49024.000000,49024.000000,49024.000000,49024.000000,49024.000000,49024.000000,49024.000000,49024.000000
mean,6.504587,2.606023,3.082449,2.545202,2.201044,0.850971,2.112071,13.797956,11.787006,0.556952,...,28109.794998,19114.742269,5397.045631,3597.975339,0.682380,0.371308,0.606907,0.606907,0.620696,99.710345
std,3.454971,1.451819,0.835960,1.490752,1.102174,1.320290,0.932532,4.625057,4.687500,0.496751,...,5185.157575,4098.891804,1213.264325,808.806094,0.465556,0.483160,0.488442,0.488442,0.485219,29.585347
min,0.510000,0.163200,1.000000,0.000000,0.000000,0.000000,0.500000,6.000000,3.050000,0.000000,...,20319.000000,13305.000000,2452.000000,1635.000000,0.000000,0.000000,0.000000,0.000000,0.000000,50.790000
25%,3.780000,1.488000,3.000000,1.000000,1.000000,0.000000,1.400000,9.700000,7.710000,0.000000,...,23598.000000,15337.000000,4746.000000,3164.000000,0.000000,0.000000,0.000000,0.000000,0.000000,73.050000
50%,5.910000,2.373000,3.000000,3.000000,2.000000,0.000000,2.130000,13.600000,11.500000,1.000000,...,30268.000000,20141.000000,5062.000000,3375.000000,1.000000,0.000000,1.000000,1.000000,1.000000,97.450000
75%,8.610000,3.468000,4.000000,4.000000,3.000000,1.000000,2.780000,17.800000,16.000000,1.000000,...,30797.000000,22123.000000,6393.000000,4262.000000,1.000000,1.000000,1.000000,1.000000,1.000000,125.567500
max,22.920000,9.726500,6.000000,5.000000,4.000000,5.000000,3.980000,21.900000,20.800000,1.000000,...,39696.000000,30351.000000,9184.000000,6122.000000,1.000000,1.000000,1.000000,1.000000,1.000000,149.750000


In [53]:
a = data['promotion_name'].unique()
print(len(a))
print(a)

49
['Cash Register Lottery' 'Double Down Sale' 'Big Time Savings'
 'Dollar Days' 'Two Day Sale' 'Super Duper Savers' 'Weekend Markdown'
 'Dollar Cutters' 'Sales Galore' 'Big Promo' 'Price Savers' 'Free For All'
 'Savings Galore' 'Unbeatable Price Savers' 'Price Slashers'
 'Price Smashers' 'Shelf Clearing Days' 'Sales Days' 'Go For It'
 'Super Savers' 'Wallet Savers' 'Save-It Sale' 'Price Destroyers'
 'Two for One' 'Big Time Discounts' 'Shelf Emptiers' 'Bye Bye Baby'
 'One Day Sale' 'Fantastic Discounts' 'Saving Days' 'High Roller Savings'
 'Sale Winners' 'Coupon Spectacular' 'Three for One' 'Price Winners'
 'Green Light Days' 'Dimes Off' 'I Cant Believe It Sale' 'Money Savers'
 'Green Light Special' 'Price Cutters' 'Tip Top Savings' 'Bag Stuffers'
 'Best Savings' 'Pick Your Savings' 'Double Your Savings' 'You Save Days'
 'Mystery Sale' 'Super Wallet Savers']


In [54]:
test = pd.read_csv('/kaggle/input/midterm-media-cost-T/test.csv',)
test.pop('id')
b = test['promotion_name'].unique()
print(len(b))
print(b)

20
['Bag Stuffers' 'High Roller Savings' 'Green Light Days' 'Price Savers'
 'Price Slashers' 'You Save Days' 'Free For All' 'Weekend Markdown'
 'Big Time Discounts' 'Shelf Clearing Days' 'Super Duper Savers'
 'Price Destroyers' 'Money Savers' 'Save-It Sale' 'I Cant Believe It Sale'
 'Go For It' 'Saving Days' 'Super Wallet Savers' 'Mystery Sale'
 'Sales Days']


In [55]:
values_to_remove = set(a) - set(b)
print(values_to_remove)
# 刪除 data 中含有 values_to_remove 的列
data_filter = data[~data['promotion_name'].isin(values_to_remove)]

{'Two Day Sale', 'Double Down Sale', 'Double Your Savings', 'Big Time Savings', 'Price Cutters', 'Super Savers', 'Wallet Savers', 'Savings Galore', 'Sale Winners', 'Dollar Days', 'Sales Galore', 'Dollar Cutters', 'Price Smashers', 'Three for One', 'Bye Bye Baby', 'Tip Top Savings', 'Unbeatable Price Savers', 'Big Promo', 'Coupon Spectacular', 'Price Winners', 'Cash Register Lottery', 'Green Light Special', 'Two for One', 'Fantastic Discounts', 'Shelf Emptiers', 'Best Savings', 'Dimes Off', 'Pick Your Savings', 'One Day Sale'}


In [56]:
data_filter.shape
c = data_filter['promotion_name'].unique()
print(c)
print(len(c))
data_filter.info()

['Super Duper Savers' 'Weekend Markdown' 'Price Savers' 'Free For All'
 'Price Slashers' 'Shelf Clearing Days' 'Sales Days' 'Go For It'
 'Save-It Sale' 'Price Destroyers' 'Big Time Discounts' 'Saving Days'
 'High Roller Savings' 'Green Light Days' 'I Cant Believe It Sale'
 'Money Savers' 'Bag Stuffers' 'You Save Days' 'Mystery Sale'
 'Super Wallet Savers']
20
<class 'pandas.core.frame.DataFrame'>
Int64Index: 17930 entries, 11 to 49023
Data columns (total 39 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   food_category             17930 non-null  object 
 1   food_department           17930 non-null  object 
 2   food_family               17930 non-null  object 
 3   store_sales(in millions)  17930 non-null  float64
 4   store_cost(in millions)   17930 non-null  float64
 5   unit_sales(in millions)   17930 non-null  int64  
 6   promotion_name            17930 non-null  object 
 7   sales_country             17

In [57]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   promotion_name  56 non-null     object 
 1   sales_country   56 non-null     object 
 2   store_type      56 non-null     object 
 3   store_city      56 non-null     object 
 4   store_state     56 non-null     object 
 5   store_sqft      56 non-null     float64
 6   grocery_sqft    56 non-null     float64
 7   frozen_sqft     56 non-null     float64
 8   meat_sqft       56 non-null     float64
 9   coffee_bar      56 non-null     float64
 10  prepared_food   56 non-null     float64
 11  florist         56 non-null     float64
 12  media_type      56 non-null     object 
dtypes: float64(7), object(6)
memory usage: 5.8+ KB


In [61]:
columns_to_drop = set(data_filter.columns) - set(test.columns)
columns_to_drop.remove('cost')
print(columns_to_drop)
# 在 data 中刪除不必要的欄位
data_filter_not_col = data_filter.drop(columns=columns_to_drop)

{'member_card', 'unit_sales(in millions)', 'marital_status', 'num_children_at_home', 'avg. yearly_income', 'store_cost(in millions)', 'total_children', 'salad_bar', 'houseowner', 'food_department', 'video_store', 'avg_cars_at home(approx)', 'recyclable_package', 'food_family', 'units_per_case', 'occupation', 'food_category', 'net_weight', 'store_sales(in millions)', 'SRP', 'gross_weight', 'low_fat', 'brand_name', 'education', 'gender'}


In [62]:
data_filter_not_col.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17930 entries, 11 to 49023
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   promotion_name  17930 non-null  object 
 1   sales_country   17930 non-null  object 
 2   store_type      17930 non-null  object 
 3   store_city      17930 non-null  object 
 4   store_state     17930 non-null  object 
 5   store_sqft      17930 non-null  int64  
 6   grocery_sqft    17930 non-null  int64  
 7   frozen_sqft     17930 non-null  int64  
 8   meat_sqft       17930 non-null  int64  
 9   coffee_bar      17930 non-null  int64  
 10  prepared_food   17930 non-null  int64  
 11  florist         17930 non-null  int64  
 12  media_type      17930 non-null  object 
 13  cost            17930 non-null  float64
dtypes: float64(1), int64(7), object(6)
memory usage: 2.1+ MB


In [63]:
float_columns = test.select_dtypes(include=['float']).columns
print(float_columns)
test[float_columns] = test[float_columns].astype(int)
test.info()

Index(['store_sqft', 'grocery_sqft', 'frozen_sqft', 'meat_sqft', 'coffee_bar',
       'prepared_food', 'florist'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   promotion_name  56 non-null     object
 1   sales_country   56 non-null     object
 2   store_type      56 non-null     object
 3   store_city      56 non-null     object
 4   store_state     56 non-null     object
 5   store_sqft      56 non-null     int64 
 6   grocery_sqft    56 non-null     int64 
 7   frozen_sqft     56 non-null     int64 
 8   meat_sqft       56 non-null     int64 
 9   coffee_bar      56 non-null     int64 
 10  prepared_food   56 non-null     int64 
 11  florist         56 non-null     int64 
 12  media_type      56 non-null     object
dtypes: int64(7), object(6)
memory usage: 5.8+ KB


In [64]:
data_filter_not_col['media_types_list'] = data_filter_not_col['media_type'].str.split(', ')
test['media_types_list'] = test['media_type'].str.split(', ')
# data_filter_not_col.head(20)

In [65]:
all_media_types = set()  # 創建一個空集合用來儲存所有類別

# 遍歷 data 中的每一列，將 media_types_list 中的類別加入到 all_media_types 集合中
for media_types_list in data_filter_not_col['media_types_list']:
    all_media_types.update(media_types_list)

# 將集合轉換為列表
all_media_types_list = list(all_media_types)
print(all_media_types_list)

['Cash Register Handout', 'Bulk Mail', 'Street Handout', 'Radio', 'Daily Paper', 'Product Attachment', 'TV', 'In-Store Coupon', 'Sunday Paper']


In [66]:
# 創建一個新的欄位列表，包含所有可能的媒體類別
all_media_types = all_media_types_list  # 包含所有可能的類別

# 將 "media_type" 欄位中的字串拆分成不同的類別
# data['media_types_list'] = data['media_type'].str.split(', ')

# 將每個可能的類別轉換成一個新的二進制欄位
for media_type in all_media_types:
    data_filter_not_col[media_type] = data_filter_not_col['media_types_list'].apply(lambda x: 1 if media_type in x else 0)
    test[media_type] = test['media_types_list'].apply(lambda x: 1 if media_type in x else 0)
# 刪除原來的 "media_type" 和 "media_types_list" 欄位
data_filter_not_col.drop(['media_type', 'media_types_list'], axis=1, inplace=True)
test.drop(['media_type', 'media_types_list'], axis=1, inplace=True)
data_filter_not_col.head(5)


,promotion_name,sales_country,store_type,store_city,store_state,store_sqft,grocery_sqft,frozen_sqft,meat_sqft,coffee_bar,...,cost,Cash Register Handout,Bulk Mail,Street Handout,Radio,Daily Paper,Product Attachment,TV,In-Store Coupon,Sunday Paper
11,Super Duper Savers,USA,Deluxe Supermarket,Tacoma,WA,33858,22123,7041,4694,1,...,69.65,0,0,0,0,0,0,0,0,1
12,Weekend Markdown,USA,Deluxe Supermarket,Tacoma,WA,33858,22123,7041,4694,1,...,73.05,0,0,0,0,0,0,0,1,0
13,Super Duper Savers,USA,Deluxe Supermarket,Tacoma,WA,33858,22123,7041,4694,1,...,69.65,0,0,0,0,0,0,0,0,1
19,Weekend Markdown,USA,Deluxe Supermarket,Tacoma,WA,33858,22123,7041,4694,1,...,73.05,0,0,0,0,0,0,0,1,0
20,Price Savers,USA,Deluxe Supermarket,Tacoma,WA,33858,22123,7041,4694,1,...,115.13,0,0,1,0,0,0,0,0,0


In [67]:
test.head(5)

,promotion_name,sales_country,store_type,store_city,store_state,store_sqft,grocery_sqft,frozen_sqft,meat_sqft,coffee_bar,...,florist,Cash Register Handout,Bulk Mail,Street Handout,Radio,Daily Paper,Product Attachment,TV,In-Store Coupon,Sunday Paper
0,Bag Stuffers,USA,Deluxe Supermarket,Salem,OR,27694,18670,5415,3610,1,...,1,0,0,0,1,1,0,0,0,0
1,High Roller Savings,USA,Deluxe Supermarket,Salem,OR,27694,18670,5415,3610,1,...,1,0,0,0,1,1,0,0,0,0
2,Green Light Days,USA,Deluxe Supermarket,Salem,OR,27694,18670,5415,3610,1,...,1,0,0,0,0,1,0,0,0,0
3,Bag Stuffers,USA,Deluxe Supermarket,Salem,OR,27694,18670,5415,3610,1,...,1,0,0,0,0,0,1,0,0,0
4,Price Savers,USA,Deluxe Supermarket,Salem,OR,27694,18670,5415,3610,1,...,1,0,0,0,1,0,0,0,0,1


In [74]:
data_filter_not_col.head(5)

,promotion_name,sales_country,store_type,store_city,store_state,store_sqft,grocery_sqft,frozen_sqft,meat_sqft,coffee_bar,...,cost,Cash Register Handout,Bulk Mail,Street Handout,Radio,Daily Paper,Product Attachment,TV,In-Store Coupon,Sunday Paper
11,Super Duper Savers,USA,Deluxe Supermarket,Tacoma,WA,33858,22123,7041,4694,1,...,69.65,0,0,0,0,0,0,0,0,1
12,Weekend Markdown,USA,Deluxe Supermarket,Tacoma,WA,33858,22123,7041,4694,1,...,73.05,0,0,0,0,0,0,0,1,0
13,Super Duper Savers,USA,Deluxe Supermarket,Tacoma,WA,33858,22123,7041,4694,1,...,69.65,0,0,0,0,0,0,0,0,1
19,Weekend Markdown,USA,Deluxe Supermarket,Tacoma,WA,33858,22123,7041,4694,1,...,73.05,0,0,0,0,0,0,0,1,0
20,Price Savers,USA,Deluxe Supermarket,Tacoma,WA,33858,22123,7041,4694,1,...,115.13,0,0,1,0,0,0,0,0,0


In [75]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import numpy as np

X = data_filter_not_col.drop('cost', axis=1,)
y = data_filter_not_col['cost']
sub_test = test.copy()
label_encoder = LabelEncoder()
for column in X.columns:
    if X[column].dtype == 'object':
        X[column] = label_encoder.fit_transform(X[column])
print(X.head(5))

for column in sub_test.columns:
    if sub_test[column].dtype == 'object':
        sub_test[column] = label_encoder.fit_transform(sub_test[column])
print(sub_test.head(5))
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=7,
                                                    test_size=0.2)
print(X_train.shape)
print(y_train.shape)

    promotion_name  sales_country  store_type  store_city  store_state  \
11              16              2           0          12            5   
12              18              2           0          12            5   
13              16              2           0          12            5   
19              18              2           0          12            5   
20              10              2           0          12            5   

    store_sqft  grocery_sqft  frozen_sqft  meat_sqft  coffee_bar  ...  \
11       33858         22123         7041       4694           1  ...   
12       33858         22123         7041       4694           1  ...   
13       33858         22123         7041       4694           1  ...   
19       33858         22123         7041       4694           1  ...   
20       33858         22123         7041       4694           1  ...   

    florist  Cash Register Handout  Bulk Mail  Street Handout  Radio  \
11        1                      0          

In [102]:
from sklearn.metrics import mean_squared_error
# model = XGBRegressor(learning_rate=0.01,
#                      n_estimators=2000,
#                      max_depth=10,
#                      random_state=42)
model = XGBRegressor(learning_rate=0.016972468699954325,
                       n_estimators=2000,
                       max_depth=9,
                       min_child_weight=0,
                       subsample=0.8,
                       colsample_bytree=0.6,
                       reg_alpha=0.4778795569577864,
                       reg_lambda=0.546751795817796,
                       objective='reg:squarederror',
                       nthread=-1,
                       random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Calculate RMSE
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

rmse_train, rmse_test

(0.3702618738823192, 0.45552515046132164)

In [95]:
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor(n_estimators=3000,
                                learning_rate=0.01,
                                max_depth=7,
#                                 min_samples_split=10,
                                random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Calculate RMSE
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

rmse_train, rmse_test

(0.3702507405156622, 0.4560942703458336)

In [103]:
preds = model.predict(sub_test)

In [104]:
output = pd.DataFrame({'cost': preds.squeeze()})
output.head()
output.to_csv('/kaggle/working/submission.csv')